In [ ]:
!pip install beautifulsoup4
!pip install gspread
!pip install google
!pip install gspread-dataframe

In [84]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime

import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
import altair as alt


In [7]:
url = 'https://scraping-for-beginner.herokuapp.com/udemy'
requests.get(url)
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [17]:
n_subscriber = soup.find('p', {'class':'subscribers'}).text
n_subscriber = int(n_subscriber.split('：')[1])

n_review = soup.find('p', {'class':'reviews'}).text
n_review = int(n_review.split('：')[1])

In [ ]:
url_ec = 'https://scraping.official.ec/'
res = requests.get(url_ec)
soup = BeautifulSoup(res.text, 'html.parser')
item_list = soup.find('ul', {'id':'itemList'})
items = item_list.find_all('li')

item = items[0]
title =  item.find('p', {'class': 'item-grid_itemTitleText_b58666da'}).text
price = item.find('p', {'class': 'items-grid_price_b58666da'}).text
price = int(price.replace('¥', '').replace(',', ''))
link = item.find('a')['href']
is_stock = item.find('p', {'class': 'items-grid_soldOut_b58666da'}) == None
is_stock = '在庫あり' if is_stock == True else '在庫なし'

AttributeError: 'NoneType' object has no attribute 'find_all'

In [30]:
url_ec = 'https://scraping.official.ec/'
res = requests.get(url_ec)
soup = BeautifulSoup(res.text, 'html.parser')
item_list = soup.find('ul', {'id':'itemList'})
items = item_list.find_all('li')

data_ec = []
for item in items:
    datum_ec = {}
    datum_ec['title'] = item.find('p', {'class': 'item-grid_itemTitleText_b58666da'}).text
    price = item.find('p', {'class': 'items-grid_price_b58666da'}).text
    datum_ec['price'] = int(datum_ec['price'].replace('¥', '').replace(',', ''))
    datum_ec['link'] = item.find('a')['href']
    is_stock = item.find('p', {'class': 'items-grid_soldOut_b58666da'}) == None
    datum_ec['is_stock'] = '在庫あり' if datum_ec['is_stock'] == True else '在庫なし'
    data_ec.append(datum_ec)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [32]:
def get_data_udemy():
    url = 'https://scraping-for-beginner.herokuapp.com/udemy'
    requests.get(url)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    n_subscriber = soup.find('p', {'class':'subscribers'}).text
    n_subscriber = int(n_subscriber.split('：')[1])

    n_review = soup.find('p', {'class':'reviews'}).text
    n_review = int(n_review.split('：')[1])

    return {
        'n_subscriber': n_subscriber,
        'n_review': n_review
    }

In [36]:
def get_data_ec(item):
    url_ec = 'https://scraping.official.ec/'
    res = requests.get(url_ec)
    soup = BeautifulSoup(res.text, 'html.parser')
    item_list = soup.find('ul', {'id':'itemList'})
    items = item_list.find_all('li')

    data_ec = []
    for item in items:
        datum_ec = {}
        datum_ec['title'] = item.find('p', {'class': 'item-grid_itemTitleText_b58666da'}).text
        price = item.find('p', {'class': 'items-grid_price_b58666da'}).text
        datum_ec['price'] = int(datum_ec['price'].replace('¥', '').replace(',', ''))
        datum_ec['link'] = item.find('a')['href']
        is_stock = item.find('p', {'class': 'items-grid_soldOut_b58666da'}) == None
        datum_ec['is_stock'] = '在庫あり' if datum_ec['is_stock'] == True else '在庫なし'
        data_ec.append(datum_ec)
    df_ec = pd.DataFrame(data_ec)

    return df_ec

In [85]:
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    'service_account.json',
    scopes=scopes
)

gc = gspread.authorize(credentials)

In [47]:
SP_SHEET_KEY =  '1GUQltcAVjUPJLUYSJ4edJ-k_A33fSJqu75x3f6A8aoI'
sh =gc.open_by_key(SP_SHEET_KEY)
SP_SHEET = 'db'
worksheet = sh.worksheet(SP_SHEET)
data = worksheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])

In [ ]:
data_udemy = get_data_udemy()
today = datetime.date.today().strftime('%Y/%m/%d')
data_udemy['date'] = today

df = pd.concat([df, pd.DataFrame([data_udemy])], ignore_index=True)
set_with_dataframe(worksheet, df, row=1, col=1)

,date,n_subscriber,n_review
0,2021/03/09,11032,1882
1,2021/03/10,11049,1885
2,2021/03/11,11068,1889
3,2021/03/12,11070,1891
4,2021/03/13,11074,1894
...,...,...,...
625,2022/11/26,17886,3093
626,2022/11/27,17902,3096
627,2022/11/28,17934,3097
628,2022/11/29,17959,3100


In [ ]:
data = worksheet.get_all_values()
df_udemy = pd.DataFrame(data[1:], columns=data[0])

,date,n_subscriber,n_review
0,2021/03/09,11032,1882
1,2021/03/10,11049,1885
2,2021/03/11,11068,1889


In [78]:
df_udemy = df_udemy.astype({
    'n_subscriber': 'int',
    'n_review': 'int'
})

ymin1 = df_udemy['n_subscriber'].min() -10 
ymax1 = df_udemy['n_subscriber'].max() + 10

ymin2 = df_udemy['n_review'].min() -10
ymax2 = df_udemy['n_review'].max() + 10

In [81]:


base = alt.Chart(df_udemy).encode(
    alt.X('date:T', axis=alt.Axis(title=None)),
)

line1 = base.mark_line().encode(
    alt.Y('n_subscriber', 
          axis=alt.Axis(title='受講生数', titleColor='#57A44C'),
          scale=alt.Scale(domain=[ymin1, ymax1]))
)

line2 = base.mark_line().encode(
    alt.Y('n_review', 
          axis=alt.Axis(title='レビュー数', titleColor='#5276A7'),
          scale=alt.Scale(domain=[ymin2, ymax2]))
)

chart = alt.layer(line1, line2).resolve_scale(
    y='independent'
)